In [1]:
import numpy as np
import os
import re

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
device = 'cuda'

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from transformers import BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import ast

In [5]:
dtype = torch.bfloat16
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=dtype,
)

In [6]:
# model_id = "codellama/CodeLlama-7b-Python-hf"
model_id = "codellama/CodeLlama-7b-Instruct-hf"

llm_tokenizer = AutoTokenizer.from_pretrained(model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto",       
    quantization_config=bnb_config,         
).eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
context = ''' 
def f(a, b):
    if a > b:
        return a
    else:
        return b
'''

context = '''
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import difflib

# Sample DataFrame setup
data = {
    'id': [1, 2, 3],
    'text': ['This is a sample text.', 'This is another example text.', 'Here is a completely different text.']
}
df = pd.DataFrame(data)

# Function to display the selected texts and highlight differences
def display_texts_and_diff(id1, id2):
    text1 = df.loc[df['id'] == id1, 'text'].values[0]
    text2 = df.loc[df['id'] == id2, 'text'].values[0]
    
    # Compute the differences
    d = difflib.Differ()
    diff = list(d.compare(text1.split(), text2.split()))
    diff_html = '<table><tr><td>{}</td><td>{}</td></tr>'.format(
        ' '.join('<span style="background-color: #ffcccc">{}</span>'.format(word.strip('-+')) if word.startswith('-') else word for word in diff if not word.startswith('+')),
        ' '.join('<span style="background-color: #ccffcc">{}</span>'.format(word.strip('-+')) if word.startswith('+') else word for word in diff if not word.startswith('-'))
    )
    
    display(HTML(diff_html))

# Create dropdowns for ID selection
id_dropdown1 = widgets.Dropdown(options=df['id'], description='ID 1:')
id_dropdown2 = widgets.Dropdown(options=df['id'], description='ID 2:')

# Button to trigger the display function
button = widgets.Button(description="Compare Texts")

# Display widgets
display(id_dropdown1, id_dropdown2, button)

# Button click event handler
def on_button_clicked(b):
    display_texts_and_diff(id_dropdown1.value, id_dropdown2.value)

button.on_click(on_button_clicked)
'''

question = 'Describe what the code is doing in one sentence'

prompt = f"""You are given the <CONTEXT/> of python code and a <QUESTION/>. ANSWER the QUESTION.
<CONTEXT>
{context}
</CONTEXT>
<QUESTION>
{question}
</QUESTION>
<ANSWER>
"""

In [7]:
model_input = llm_tokenizer(prompt, return_tensors="pt").to('cuda')
n_tokens = model_input['input_ids'].shape[1]

In [9]:
from tqdm import tqdm
# for _ in tqdm(range(1000)):
    # with torch.no_grad():
output = llm_model.generate(**model_input, max_new_tokens=300, pad_token_id=llm_tokenizer.eos_token_id)

In [10]:
output = output[0, n_tokens:]
output = llm_tokenizer.decode(output, skip_special_tokens=True)
output = output.split('</ANSWER>')[0].strip()

In [11]:
print(output)

The code is creating a dropdown menu for selecting two IDs from a dataframe, and then displaying the corresponding text values side-by-side with highlighted differences.


In [7]:
def answer(question, max_new_tokens=300):
    model_input = llm_tokenizer(question, return_tensors="pt").to('cuda')
    n_tokens = model_input['input_ids'].shape[1]
    output = llm_model.generate(**model_input, max_new_tokens=max_new_tokens, pad_token_id=llm_tokenizer.eos_token_id)
    output = output[0, n_tokens:]
    output = llm_tokenizer.decode(output, skip_special_tokens=True)
    return output



In [9]:

q = "Create an array of length 5 which contains all even numbers between 1 and 10."
print(answer(q))




```js
const evenNumbers = [2, 4, 6, 8, 10];
```

### Solution

```js
const evenNumbers = [];

for (let i = 2; i <= 10; i += 2) {
  evenNumbers.push(i);
}
```



In [10]:
print(answer("Formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length."))



Comment: I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this question as off-topic because it is about mathematics.

Comment: @MatthewLeingang I'm voting to close this q

In [11]:
print(answer("Write a function to generate the nth Fibonacci number"))

.

The Fibonacci numbers are the numbers in the following integer sequence:

0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, ...

For example, `fib(4)` should return 3.

### Solution

```js
function fib(n) {
  if (n <= 1) return n;
  return fib(n - 1) + fib(n - 2);
}
```

### Tests

```js
const { assert } = require('chai');

describe('fib(n)', () => {
  it('returns the nth Fibonacci number', () => {
    assert.strictEqual(fib(0), 0);
    assert.strictEqual(fib(1), 1);
    assert.strictEqual(fib(2), 1);
    assert.strictEqual(fib(3), 2);
    assert.strictEqual(fib(4), 3);
    assert.strictEqual(fib(5), 5);
    assert.strictEqual(fib(6), 8


In [12]:
# read a json file
import json
with open('data/code_alpaca_20k.json') as f:
    questions = json.load(f)

In [15]:
from tqdm import tqdm

for q in tqdm(questions):
    print(q['instruction'])
    print(answer(q['instruction']))

  3%|▎         | 644/20022 [7:53:18<217:46:02, 40.46s/it]




### Learnings from the project

 After completing this project, you will have a better understanding of how to build a deep learning model. In this project, you will apply the following concepts.

- Data Preprocessing
- Data Normalization
- Building a deep learning model
- Model evaluation


### Approach taken to solve the problem

 The dataset was preprocessed and split into training and testing sets. The model was trained on the training set and then evaluated on the testing set.


### Challenges faced

 The dataset was not clean and had missing values.


### Additional pointers

 - The dataset was not clean and had missing values.



Write an R script to find the correlation between two variables.


  3%|▎         | 645/20022 [7:54:06<229:33:00, 42.65s/it]



I have a data set with two variables, `x` and `y`. I want to find the correlation between these two variables.

I have tried the following code:

\begin{code}
library(corrplot)

x <- c(1,2,3,4,5,6,7,8,9,10)
y <- c(1,2,3,4,5,6,7,8,9,10)

correlation <- cor(x,y)

corrplot(correlation)
\end{code}

However, I get the following error:

\begin{code}
Error in cor(x, y) : 'x' must be numeric
\end{code}

I have tried to convert the variables to numeric, but I still get the same error.

\begin{code}
x <- as.numeric(c(1,2,3,4,5,6,7,8,9,10))
y <- as.numeric(c(1,2,3,4,5,6,7,8,9,10))
\end{code}

Comment: `x` and `y` are already numeric. You can check with `str(x)` and `str(y)`.

Comment: @RuiBarr
Write a JavaScript function to sort an array of integers.


  3%|▎         | 646/20022 [7:54:48<229:08:39, 42.57s/it]

...



\begin{code}
public class Test {
    public static void main(String[] args) {
        int[][] a = new int[3][];
        a[0] = new int[3];
        a[1] = new int[3];
        a[2] = new int[3];
        a[0][0] = 1;
        a[1][0] = 2;
        a[2][0] = 3;
        System.out.println(a[0][0]);
        System.out.println(a[1][0]);
        System.out.println(a[2][0]);
    }
}
\end{code}

Comment: What is the problem?

Comment: @SotiriosDelimanolis I think the problem is that the OP is trying to use a 2D array as a 1D array.

Comment: @SotiriosDelimanolis I think the problem is that the OP is trying to use a 2D array as a 1D array.

Comment: @SotiriosDelimanolis I think the problem is that the OP is trying to use a 2D array as a 1D array.

Comment: @SotiriosDelimanolis I think the problem is that the OP is
To what type of data can the following MySQL query return?


  3%|▎         | 650/20022 [7:58:01<252:34:09, 46.94s/it]




\begin{code}
SELECT * FROM table_name WHERE column_name = 'value'
\end{code}

Comment: It can return any type of data that is in the column.

Comment: @Barmar: I think the question is asking about the type of data that can be stored in the column, not the type of data that can be returned by the query.

Comment: @Barmar: I think the question is asking about the type of data that can be stored in the column, not the type of data that can be returned by the query.

Comment: @Barmar: I think the question is asking about the type of data that can be stored in the column, not the type of data that can be returned by the query.

Comment: @Barmar: I think the question is asking about the type of data that can be stored in the column, not the type of data that can be returned by the query.

Comment: @Barmar: I think the question is asking about the type of data that can be stored in the column, not the type of data that can be returned by the query.

Comment: @Barmar: I think the question i

  3%|▎         | 650/20022 [7:58:34<237:42:57, 44.18s/it]


KeyboardInterrupt: 

In [9]:
print(answer("Answer the question: Was code-llama trained in a supervised or unsupervised manner?"))


 The answer is: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsupervised manner.

Answer: The code-llama was trained in an unsup